In [1]:
from main import *

/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
# toy example

train_sub = [1,2,3]
train_x0, train_y0 = make_train_dataset(train_sub, 0)
score_model0 = train_scorenet(train_x0, train_y0)

train length :  432


Score model Average Loss: 23747.645038:   8%|████████▉                                                                                                 | 421/5000 [15:45<2:52:04,  2.25s/it]

In [ ]:
samples0 = sampling(score_model0, sample_batch_size=batch_size)

generated_signal0 = return_to_signal(samples0)
